In [280]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from pigeon import annotate
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
import pickle

import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [10]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


# Labeling

In [56]:
DATA_MODES = ['train', 'test']
DEVICE = torch.device("cuda")

In [57]:
DATA_DIR = Path('EyesDataset/')
files = sorted(list(DATA_DIR.rglob('*.jpg')))

In [ ]:
from IPython.display import display, Image

In [3]:
annotations = annotate(
  files,
  options=['opened', 'closed'],
  display_fn=lambda filename: display(Image(filename))
)

HTML(value='0 examples annotated, 4001 examples left')

Output()

Annotation done.


In [6]:
with open('annotations.pkl', 'wb') as f:
    pickle.dump(annotations, f)

In [3]:
with open('annotations.pkl', 'rb') as f:
    annotations = pickle.load(f)

In [6]:
len(annotations)

3955

In [24]:
train_ann, test_ann = train_test_split(annotations, test_size=0.1, shuffle=True)

# Dataset class

In [59]:
from PIL import Image 

In [210]:
class EyesDataset(Dataset):
    def __init__(self, annotations, mode):
        super().__init__()
        # список файлов для загрузки
        self.annotations = annotations
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.annotations)
     
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
        augmentation = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomResizedCrop(24, scale=(0.8, 1), ratio=(0.75, 1.3333333333333333), interpolation=2),
#             transforms.RandomPerspective(distortion_scale=0.3, p=0.9, interpolation=3, fill=0),
            transforms.RandomAffine(degrees=20, shear=20, resample=False),
            transforms.Resize((24, 24))
        ])
        
        x = self.load_sample(self.annotations[index][0])
        label = (lambda x : 0 if (x == 'closed') else 1)(self.annotations[index][1])
        label = np.array(label, dtype='float32')
                                                         
        if self.mode == 'train':
            x = augmentation(x)
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        return x, label
        
    def _prepare_sample(self, image):
        image = image.resize((24, 24))
        return np.array(image)

In [211]:
train_dataset = EyesDataset(train_ann, 'train')
test_dataset = EyesDataset(test_ann, 'test')

In [212]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=32)

In [213]:
mobilenet = models.mobilenet_v2()

In [214]:
mobilenet.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [215]:
mobilenet.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [216]:
mobilenet.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False), 
    nn.Linear(in_features=1280, out_features=100, bias=True), 
    nn.ReLU(), 
    nn.Linear(in_features=100, out_features=1, bias=True)
)

In [248]:
def train_epoch(net, train_loader, loss_fn, optimizer):
    net.train()
    train_ep_loss = 0.
    counter = 0
    for train_x, train_y in train_loader:
        train_x = train_x.to(DEVICE)
        train_y = train_y.to(DEVICE).view(-1, 1)
        # zero  gradients
        optimizer.zero_grad()

        # get the output from the model
        output = net(train_x)
        output = torch.sigmoid(output)
        # calculate loss
        loss = loss_fn(output, train_y)
        loss.backward()
        optimizer.step()

        train_ep_loss += loss.item() 
        counter += 1
    train_ep_loss /= counter
   
    return train_ep_loss

In [261]:
def test_epoch(net, test_loader, loss_fn):
    net.eval()
    test_ep_loss = 0.
    counter = 0.
    for test_x, test_y in test_loader:
        # get the output from the model
        test_x = test_x.to(DEVICE)
        test_y = test_y.to(DEVICE).view(-1, 1)
        
        output = net(test_x)
        output = torch.sigmoid(output)
        # calculate loss
        loss = loss_fn(output, test_y)
        test_ep_loss += loss.item() 
        
        counter += 1

    test_ep_loss /= counter
#     ind = np.random.randint(len(test_x))

    return test_ep_loss

In [262]:
def train(net, train_loader, test_loader, loss_fn, optimizer, epochs, scheduler=None):
    
    train_losses = []
    test_losses = []

    for e in tqdm(range(epochs)):

        train_loss = train_epoch(net, train_loader, loss_fn, optimizer)
        if scheduler:
            scheduler.step()
        with torch.no_grad():
            test_loss = test_epoch(net, test_loader, loss_fn)

        train_losses.append(train_loss)
        test_losses.append(test_loss)

        print("Epoch: {}/{}...".format(e+1, epochs),
                      "Loss: {:.6f}...".format(train_loss),
                      "Test Loss: {:.6f}".format(test_loss))
    return train_losses, test_losses

In [263]:
mobilenet = mobilenet.to(DEVICE)

optimizer = torch.optim.Adam(mobilenet.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)
criterion = nn.BCELoss()

In [264]:
mn_train_loss, mn_test_losses = train(mobilenet, train_loader, test_loader, criterion, optimizer, 20, scheduler)

  5%|▌         | 1/20 [00:07<02:20,  7.40s/it]

Epoch: 1/20... Loss: 0.361147... Test Loss: 0.222717


 10%|█         | 2/20 [00:14<02:12,  7.37s/it]

Epoch: 2/20... Loss: 0.317524... Test Loss: 0.197885


 15%|█▌        | 3/20 [00:21<02:04,  7.31s/it]

...

Epoch: 18/20... Loss: 0.187499... Test Loss: 0.132556


 95%|█████████▌| 19/20 [02:19<00:07,  7.43s/it]

Epoch: 19/20... Loss: 0.182721... Test Loss: 0.123317


100%|██████████| 20/20 [02:27<00:00,  7.36s/it]

Epoch: 20/20... Loss: 0.198498... Test Loss: 0.127191


In [266]:
test_batch_x, test_batch_y = iter(test_loader).next()

In [279]:
test_batch_y = test_batch_y.numpy()

In [277]:
test_batch_pred = np.array((mobilenet(test_batch_x.to(DEVICE)) > 0).view(-1).cpu().detach(), dtype='float32')

In [278]:
test_batch_pred

array([0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.],
      dtype=float32)

In [281]:
accuracy_score(test_batch_y, test_batch_pred)

0.90625